# **QA Chatbot**

In [39]:
# %pip -q install langchain openai tiktoken chromadb google-search-results langchainhub
# %pip show langchain

In [40]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.document_loaders import DirectoryLoader
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory, ReadOnlySharedMemory
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain.tools import BaseTool
from langchain import SerpAPIWrapper
from langchain.utilities import SerpAPIWrapper
from langchain import hub
from langchain.agents import create_openai_functions_agent
from langchain import LLMMathChain
import os
import config

In [41]:
API_KEY = os.environ["OPENAI_API_KEY"] = config.OPENAI_API_KEY

In [42]:
# Definir constantes
CHROMA_PATH = "chroma"  # Ruta para la base de datos de Chroma
DATA_PATH = "./"  # Ruta al directorio que contiene los datos

In [43]:
file_pattern = 'embeddings.md'

## **Embeddings**

In [44]:
# Instanciación del cargador de directorio con el patrón de archivo especificado
loader = DirectoryLoader(DATA_PATH, glob=file_pattern,
                         loader_cls=lambda path: TextLoader(path, encoding='utf-8'))

# Carga de documentos utilizando el cargador
documents = loader.load()

In [45]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=200)
texts = text_splitter.split_documents(documents)

In [46]:
len(texts)

141

In [47]:
texts[0]

Document(page_content='# Plan de Estudios: Ingeniería en Datos e Inteligencia Organizacional (Plan 2016)\n\n## Créditos Totales\n\n404 créditos\n\n## Descripción General', metadata={'source': 'embeddings.md'})

In [48]:
# Directorio de persistencia para almacenar la base de datos
persist_directory = 'db'

# Seleccionamos OpenAI embeddings para la generación de vectores
embedding = OpenAIEmbeddings()

# Creamos una instancia de Chroma para generar vectores a partir de documentos
vectordb = Chroma.from_documents(
    documents=texts,  # Los documentos de texto a procesar para la generación de vectores
    embedding=embedding,  # El método de embedding a utilizar para generar vectores
    persist_directory=persist_directory  # Directorio donde se almacenará la base de datos de vectores
)

In [49]:
vectordb.persist()
vectordb = None

In [50]:
# Creación de una nueva instancia de Chroma, para cargar una base de datos existente
vectordb = Chroma(
    persist_directory=persist_directory,  # Directorio de persistencia de la base de datos de vectores
    embedding_function=embedding  # Función de embedding a utilizar
)

In [51]:
# Convertir la base de datos de vectores en un objeto retriever
retriever = vectordb.as_retriever()

In [52]:
# Usando el objeto retriever para obtener documentos relevantes para una consulta específica
docs = retriever.get_relevant_documents("Total de creditos de la carrera")

In [53]:
docs

[Document(page_content='404 créditos\n\n## Descripción General\n\nTodas las materias de la Ingeniería en Datos e Inteligencia Organizacional tienen un ciclo, una clave, un nombre, un número de créditos, un tipo y una academia.', metadata={'source': 'embeddings.md'}),
 Document(page_content='404 créditos\n\n## Descripción General\n\nTodas las materias de la Ingeniería en Datos e Inteligencia Organizacional tienen un ciclo, una clave, un nombre, un número de créditos, un tipo y una academia.', metadata={'source': 'embeddings.md'}),
 Document(page_content='# Plan de Estudios: Ingeniería en Datos e Inteligencia Organizacional (Plan 2016)\n\n## Créditos Totales\n\n404 créditos\n\n## Descripción General', metadata={'source': 'embeddings.md'}),
 Document(page_content='# Plan de Estudios: Ingeniería en Datos e Inteligencia Organizacional (Plan 2016)\n\n## Créditos Totales\n\n404 créditos\n\n## Descripción General', metadata={'source': 'embeddings.md'})]

In [54]:
len(docs)

4

In [55]:
retriever = vectordb.as_retriever(search_kwargs={"k": 2})

In [56]:
retriever.search_type

'similarity'

In [57]:
retriever.search_kwargs

{'k': 2}

In [58]:
# Creación de un objeto RetrievalQA a partir de un tipo de cadena específico
qa_chain = RetrievalQA.from_chain_type(
    llm=OpenAI(),  # Modelo de lenguaje a utilizar para la generación de respuestas
    # Tipo de cadena (podría ser una configuración específica para el QA)
    chain_type="stuff",
    retriever=retriever,  # Objeto retriever utilizado para recuperar documentos relevantes
    # Especifica si se deben devolver los documentos de origen junto con las respuestas
    return_source_documents=True
)

In [59]:
def process_llm_response(llm_response):
    # Imprimir la respuesta generada por el modelo de lenguaje
    print(llm_response['result'])

    # Imprimir la sección de fuentes de la respuesta
    print('\n\nSources:')

    # Iterar sobre cada fuente en los documentos de origen asociados con la respuesta
    for source in llm_response["source_documents"]:
        # Imprimir la fuente de cada documento de origen
        print(source.metadata['source'])

### Pruebas

In [60]:
# Consulta específica
query = "Quiero cargar Electricidad y magnetismo, ¿qué debo hacer?"

# Obtener la respuesta del modelo de pregunta-respuesta
llm_response = qa_chain(query)

# Procesar la respuesta utilizando la función process_llm_response
process_llm_response(llm_response)

 Debes cargar y aprobar primero Física Clásica antes de cargar Electricidad y Magnetismo.


Sources:
embeddings.md
embeddings.md


In [61]:
query = "Quiero  cargar Calculo Integral, que debo hacer?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 Para cargar la materia de Cálculo Integral, debes seguir los siguientes pasos: 
1. Revisar que esté en el Ciclo 1. 
2. Obtener la clave de la materia, en este caso II0209. 
3. Verificar que la materia se llame Cálculo Integral. 
4. Comprobar que tenga 6 créditos. 
5. Confirmar que sea una materia de tipo básica. 
Una vez que hayas revisado todos estos aspectos, podrás cargar la materia de Cálculo Integral en tu plan de estudios. Si tienes dudas o problemas, puedes acudir a tu asesor académico para que te oriente en el proceso. 


Sources:
embeddings.md
embeddings.md


In [62]:
query = "cuantos ciclos son?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 Existen 4 ciclos en total.


Sources:
embeddings.md
embeddings.md


In [63]:
query = "cual es la clave de la materia Pensamiento crítico para ingeniería y de que ciclo es?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 La clave de la materia es ID0160 y pertenece al ciclo 1.


Sources:
embeddings.md
embeddings.md


In [64]:
query = "¿cuantas materias y cuales debo aprobar antes de cargar Aprendizaje Estadistico?"
llm_response = qa_chain(query)
process_llm_response(llm_response)


Debes aprobar todas las materias básicas del primer ciclo y al menos 2 materias de elección libre antes de cargar Aprendizaje Estadístico. No se especifica cuántas materias básicas hay en total o cuáles son, así que no se puede dar una respuesta exacta.


Sources:
embeddings.md
embeddings.md


In [65]:
query = "¿cuantas materias y cuales debo aprobar antes de cargar Ecuaciones Diferenciales?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 Durante el primer ciclo debes cargar y aprobar todas las materias basicas y aprobar por lo menos 2 de Eleccion Libre. Por lo tanto, antes de cargar Ecuaciones Diferenciales, debes aprobar todas las materias basicas y al menos dos de Eleccion Libre durante el primer ciclo.


Sources:
embeddings.md
embeddings.md


In [66]:
query = "cuantos son los creditos totales de la carrera?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 404 créditos


Sources:
embeddings.md
embeddings.md


In [67]:
query = " Hablame de la materia de Aprendizaje Estadistico"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 La materia de Aprendizaje Estadístico es una materia básica del tercer ciclo con clave ID0309 y 6 créditos. Su objetivo es enseñar a los estudiantes sobre el uso de técnicas estadísticas para analizar datos y tomar decisiones basadas en ellos. Es una materia importante para aquellos que deseen especializarse en áreas como investigación, análisis de datos y ciencias sociales.


Sources:
embeddings.md
embeddings.md


In [68]:
query = " cual es la clave de esa materia?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 La clave de esa materia es IT0322.


Sources:
embeddings.md
embeddings.md


## **Agente: Google Search Results**

In [69]:
llm = ChatOpenAI(temperature=0, max_tokens=1000, model="gpt-3.5-turbo")

In [70]:
os.environ["SERPAPI_API_KEY"] = config.SERPAPI_API_KEY

In [71]:
search = SerpAPIWrapper()

In [72]:
prompt = hub.pull("hwchase17/openai-functions-agent")

In [73]:
prompt

ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'openai-functions-agent', 'lc_hub_commit_hash': 'a1655024b06afbd95d17449f21316291e0726f13dcfaf990cc0d18087ad689a5'}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')), MessagesPlace

## **Agente: LLM MathChain**

In [74]:
llm_math_chain = LLMMathChain.from_llm(llm=llm, verbose=True)

In [75]:
tools = [
    Tool(
        name="QA System",
        func=qa_chain.run,
        description="Útil cuando necesitas responder preguntas sobre El mapa curricular de IDeIO. La entrada debe ser una pregunta completamente formulada."
    ),

    Tool(
        name="Backup QA Google Search",
        func=search.run,
        description="Útil cuando necesitas responder preguntas sobre El mapa curricular de IDeIO, pero solo cuando el Sistema de Preguntas y Respuestas de El mapa curricular de IDeIO no pudo responder la consulta. La entrada debe ser una pregunta completamente formulada."
    ),
    Tool(
        func=llm_math_chain.run,
        name="Calculadora",
        description="Herramienta que suma las materias y suma los creditos de las materias ",
    ),
]

In [77]:
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True, prompt=prompt)

In [81]:
agent.run("Cuantos creditos son en total?")



> Entering new AgentExecutor chain...
I need to calculate the total number of credits in the curriculum.
Action: Calculadora
Action Input: Sumar los créditos de todas las materias

> Entering new LLMMathChain chain...
Sumar los créditos de todas las materias```text
25 + 20 + 15 + 30 + 10
```
...numexpr.evaluate("25 + 20 + 15 + 30 + 10")...

Answer: 100
> Finished chain.

Observation: Answer: 100
Thought:I have calculated the total number of credits in the curriculum.
Final Answer: 100 credits

> Finished chain.


'100 credits'

## **Process LLM**

In [82]:
def process_llm_response(llm_response):
    # Imprimir la respuesta generada por el modelo de lenguaje
    print(llm_response['result'])

    # Imprimir la sección de fuentes de la respuesta
    print('\n\nSources:')

    # Iterar sobre cada fuente en los documentos de origen asociados con la respuesta
    for source in llm_response["source_documents"]:
        # Imprimir la fuente de cada documento de origen
        print(source.metadata['source'])

In [83]:
query = "Quiero  cargar Calculo Integral, que debo hacer?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 Debes buscar la materia con clave II0209 en el ciclo 1 y asegurarte de que tenga 6 créditos y sea de tipo básica. Luego, debes inscribirte en ella para poder cursarla.


Sources:
embeddings.md
embeddings.md


In [84]:
query = "cuantos ciclos son?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 
Hay 4 ciclos: Ciclo 1, Ciclo 2, Ciclo 3 y Ciclo 4.


Sources:
embeddings.md
embeddings.md


In [85]:
query = "cual es la clave de la materia Pensamiento crítico para ingeniería y de que ciclo es?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 La clave de la materia es ID0160 y pertenece al ciclo 1.


Sources:
embeddings.md
embeddings.md


In [86]:
query = "¿cuantas materias y cuales debo aprobar antes de cargar Aprendizaje Estadistico?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 Debes aprobar todas las materias básicas del primer ciclo y al menos dos materias de elección libre antes de cargar Aprendizaje Estadístico en el segundo ciclo.


Sources:
embeddings.md
embeddings.md
